In [1]:
import grammar as g
import tmp as t
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import importlib

/home/alexserra98/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Grammar

In [2]:
gramm = g.Grammar('/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/cnn.grammar.txt')


In [5]:
for i in gramm.grammar:
    print(f'{i} : {gramm.grammar[i]} \n')

features : [[('convolution', True), ('pooling', True)], [('convolution', True)], [('pooling', True)], [('pooling', True)], [('dropout', True)], [('batch-norm', True)]] 

convolution : [[('layer:conv', False), ('[num-filters,int,1,32,256]', False), ('[filter-shape,int,1,2,5]', False), ('[stride,int,1,1,3]', False), ('padding', True), ('activation-function', True), ('bias', True)]] 

batch-norm : [[('layer:batch-norm', False)]] 

pooling : [[('pool-type', True), ('[kernel-size,int,1,2,5]', False), ('[stride,int,1,1,3]', False), ('padding', True)]] 

pool-type : [[('layer:pool-avg', False)], [('layer:pool-max', False)]] 

padding : [[('padding:same', False)], [('padding:valid', False)]] 

dropout : [[('layer:dropout', False), ('[rate,float,1,0,0.7]', False)]] 

classification : [[('fully-connected', True)], [('dropout', True)]] 

fully-connected : [[('layer:fc', False), ('activation-function', True), ('[num-units,int,1,128,2048]', False), ('bias', True)]] 

activation-function : [[('act:l

In [6]:
prova = gramm.initialise('features')

In [7]:
for i in prova:
    print(f'{i} : {prova[i]}')

features : [{'ge': 2, 'ga': {}}]
pooling : [{'ge': 0, 'ga': {'kernel-size': ('int', 2.0, 5.0, [3]), 'stride': ('int', 1.0, 3.0, [1])}}]
pool-type : [{'ge': 0, 'ga': {}}]
padding : [{'ge': 1, 'ga': {}}]


### Phenotype

In [8]:
feat_layers = gramm.decode('features', prova)

In [9]:
layer_decoding = t.Evaluator()
l_dict = layer_decoding.get_layers(feat_layers) 
l_dict

[('pool-avg', {'kernel-size': ['3'], 'stride': ['1'], 'padding': ['valid']})]

In [10]:
feat_layers

'layer:pool-avg kernel-size:3 stride:1 padding:valid'

In [11]:
type(l_dict)

list

In [35]:
importlib.reload(t)

<module 'tmp' from '/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/tmp.py'>

In [14]:

assembler = t.Evaluator()
lay = assembler.assemble_network(l_dict,(28,28,3))

layer_type: pool-avg


In [15]:
type(lay)

torch.nn.modules.container.Sequential

### Assembling Layers

In [32]:
importlib.reload(t)

<module 'tmp' from '/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/tmp.py'>

In [3]:
gramm = g.Grammar('/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/cnn.grammar.txt')

#### feature module

In [4]:
my_module = t.Module('features',1,10,)
init_max = {'features' : 5,'classification' : 1, 'learning' : 1}
my_module.initialise(gramm,0.2,init_max)


In [7]:
my_layers_decoded = []
for i in my_module.layers:
    my_layers_decoded.append(gramm.decode('features', i))

#### classification module

In [6]:
my_module1 = t.Module('classification',1,2)
init_max = {'features' : 5,'classification' : 1, 'learning' : 1}
my_module1.initialise(gramm,0.2,init_max)

In [40]:
my_module1.layers


[{'classification': [{'ge': 0, 'ga': {}}],
  'fully-connected': [{'ge': 0,
    'ga': {'num-units': ('int', 128.0, 2048.0, [992])}}],
  'activation-function': [{'ge': 0, 'ga': {}}],
  'bias': [{'ge': 0, 'ga': {}}]}]

In [8]:
my_layers_decoded_1 = []
for i in my_module1.layers:
    my_layers_decoded_1.append(gramm.decode('classification', i))

In [9]:
pheno_class =' '.join(my_layers_decoded_1)
pheno_feat =' '.join(my_layers_decoded)

In [36]:
layer_decoding = t.Net_encoding(5,2,3,2,(28,28,3),pheno_feat,pheno_class)

In [11]:

my_layer_list = layer_decoding.get_layers(pheno_class)

In [12]:
my_layer_list

[('fc', {'act': ['tanh'], 'num-units': ['992'], 'bias': ['True']})]

In [37]:
lay,size = layer_decoding.assemble_block(my_layer_list,2352)

layer_type: fc


In [23]:
size 

992

#### learning module

In [ ]:
my_module2 = t.Module('learning',1,2,)
init_max = {'features' : 5,'classification' : 1, 'learning' : 1}
my_module2.initialise(gramm,0.2,init_max)

In [40]:
last_my_layers = my_layers_decoded + my_layers_decoded_1
phenotype =' '.join(last_my_layers)

In [42]:
my_layer_list = layer_decoding.get_layers(phenotype)

In [38]:
lay = layer_decoding.assemble_model(pheno_feat,pheno_class)

layer_type: batch-norm
layer_type: conv
layer_type: dropout
layer_type: dropout
layer_type: conv
layer_type: fc


In [39]:
lay

genNN(
  (fe): Sequential(
    (0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Conv2d(3, 203, kernel_size=(2, 2), stride=(1, 1), padding=valid)
    (2): Sigmoid()
    (3): Dropout2d(p=0.4968258204193914, inplace=False)
    (4): Dropout2d(p=0.4968258204193914, inplace=False)
    (5): Conv2d(203, 203, kernel_size=(2, 2), stride=(1, 1), padding=valid)
    (6): Sigmoid()
  )
  (c): Sequential(
    (0): Linear(in_features=116928, out_features=992, bias=True)
    (1): Tanh()
  )
  (last): Sequential(
    (0): Linear(in_features=992, out_features=10, bias=True)
  )
)

In [57]:
class LeNet5(nn.Module):

    def __init__(self, n_classes,fe,c):
        super(LeNet5, self).__init__()
        
    def forward(self, x):
        x = self.fe(x)
        x = torch.flatten(x, 1)
        logits = self.c(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs

feature_extractor = nn.Sequential(            
    nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
    nn.Tanh(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
    nn.ReLU()
)

classifier = nn.Sequential(
    nn.Linear(in_features=120, out_features=84),
    nn.ReLU(),
    nn.Linear(in_features=84, out_features=10),
)

In [60]:
my_object = LeNet5(10,feature_extractor,classifier)